In [1]:
!echo Installing bbSearch module from web ...
!echo creating bbmodcache subfolder
!mkdir -p bbmodcache
!echo downloading bbSearch module
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py
!pip install matplotlib

from bbSearch import SearchProblem, search
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from copy import deepcopy
from enum import Enum
import sys

Installing bbSearch module from web ...
creating bbmodcache subfolder


A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file bbmodcache already exists.
Error occurred while processing: bbmodcache.


downloading bbSearch module


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 18767  100 18767    0     0   241k      0 --:--:-- --:--:-- --:--:--  251k

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: C:\Users\lukew\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Loading bbSearch Version 2.1 (at 14:12, Fri 21 Feb)
Last module source code edit 9am Thursday 24th Feb 2022


# B - Robot Worker

In [2]:
class Item(Enum):
    RUSTY_KEY = "Rusty Key"
    BUCKET = "Bucket"
    SUITCASE = "Suitcase"
    SCREWDRIVER = "Screwdriver"
    SLEDGE_HAMMER = "Sledge Hammer"
    ANVIL = "Anvil"
    SAW = "Saw"
    GOLD_BAR = "Gold Bar"
    SILVER_BAR = "Silver Bar"
    DIAMOND_RING = "Diamond Ring"
    OVEN_MITS = "Oven Mits"
    HOT_SOUP = "Hot Soup"
    LASAGNA = "Lasagna"

class Room(Enum):
    WORKSHOP = "Workshop"
    STORE_ROOM = "Store Room"
    TOOL_CUPBOARD = "Tool Cupboard"
    KITCHEN = "Kitchen"
    BEDROOM = "Bedroom"
    CHARGING_ROOM = "Charging Room"


In [3]:
class Robot:
    def __init__(self, location : Room, carried_items : list[Item], strength : int, max_charge : int, initial_charge : int):
        self.location      = location
        self.carried_items = carried_items
        self.strength      = strength
        self.battery = min(max_charge, initial_charge)
        self.max_charge = max_charge

    def weight_carried(self):
        return sum([ITEM_WEIGHTS[i] for i in self.carried_items])
    
    def value_carried(self):
        return sum([ITEM_VALUES[i] for i in self.carried_items])

    def charge_up(self):
        self.battery = self.max_charge
    
    def reduce_charge(self, amount):
        self.battery -= amount
        self.battery = max(0, self.battery)

    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location,
                      self.carried_items,
                      self.strength,
                      self.battery ) )

class Door:
    def __init__(self, roomA : Room, roomB : Room, doorkey : Item = None, locked : bool =False):
        self.goes_between = { roomA, roomB }
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}

    ## Define a unique string representation for a door object
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )

In [4]:
class GoalParam:
    """
    Specifies a set of items that are required to be in a room
    and the minimum value of all the items in the room
    """
    
    def __init__(self, room : Room, contents : set[Item] = set(), desired_value : int = -sys.maxsize):
        self.room = room
        self.contents = contents
        self.desired_value = desired_value

In [5]:
class State:
    def __init__(self, robot : Robot, doors : list[Door], room_contents : dict[Room, set[Item]]):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents
        self.goal_params : set[GoalParam] = set()

    # Calculates the value of a room's contents
    def room_value(self, room : Room) -> int:
        return sum(ITEM_VALUES[i] for i in self.room_contents[room])

    ## Define a string representation that will be uniquely identify the state.
    ## An easy way is to form a tuple of representations of the components of
    ## the state, then form a string from that:
    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )

In [6]:
class Action(Enum):
    CHANGE_ROOM = "Change room to"
    PICK_UP = "Pick up"
    PUT_DOWN = "Put down"

In [7]:
class RobotWorker( SearchProblem ):
    def __init__( self, state : State, goal_params : set[GoalParam] ):
        self.initial_state = state
        self.initial_state.goal_params = goal_params
        self.goal_params = goal_params

    def possible_actions(self, state: State) -> list[tuple[Action, Item | Room]]:
        robot_location = state.robot.location
        strength       = state.robot.strength
        charge         = state.robot.battery
        weight_carried = state.robot.weight_carried()

        if charge < 1:
            return []

        actions : list[tuple[Action, Item | Room]] = []
        # Can put down any carried item
        for item in state.robot.carried_items:
            # If item has dependents
            if item in DEPENDENT_ITEMS:
                dependent_items = DEPENDENT_ITEMS[item]
                # Check if any dependent item is in inventory
                valid = True
                for dependent in dependent_items:
                    if dependent in state.robot.carried_items:
                        valid = False
                        break
                if valid:
                    actions.append((Action.PUT_DOWN, item))
            else:
                actions.append((Action.PUT_DOWN, item))


        # Can pick up any item in room if strong enough and has item it depends on
        for item in state.room_contents[robot_location]:
            if strength >= weight_carried + ITEM_WEIGHTS[item]:
                # If item has required items
                if item in REQUIRED_ITEMS:
                    for required_item in REQUIRED_ITEMS[item]:
                        if required_item in state.robot.carried_items:
                            actions.append((Action.PICK_UP, item))
                else:
                    actions.append((Action.PICK_UP, item))


        # If there is an unlocked door between robot location and
        # another location can move to that location
        for door in state.doors:
            if door.locked == False and robot_location in door.goes_between:
                actions.append((Action.CHANGE_ROOM, door.other_loc[robot_location]))

        # Now the actions list should contain all possible actions
        return actions

    def successor( self, state : State, action : tuple[Action, Item | Room]):
        next_state = deepcopy(state)
        act, target = action
        next_state.robot.reduce_charge(1)

        if act == Action.PUT_DOWN:
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)

        if act == Action.PICK_UP:
            next_state.robot.carried_items.append(target)
            next_state.room_contents[state.robot.location].remove(target)

        if act == Action.CHANGE_ROOM:
            next_state.robot.location = target
            if (next_state.robot.location == Room.CHARGING_ROOM):
                next_state.robot.charge_up()

        return next_state

    def goal_test(self, state : State):
        #print(state.room_contents)
        for goal_param in self.goal_params:
            room = goal_param.room
            contents = goal_param.contents
            for i in contents:
                if not i in state.room_contents[room]:
                    return False

            # Check desired_value is greater (or equal) to the value in the room
            desired_value = goal_param.desired_value
            actual_value = state.room_value(room)
            if actual_value < desired_value:
                return False

        return True

    def display_state(self, state : State):
        print("Robot location:", state.robot.location.value)
        print("Robot carrying:", [item.value for item in state.robot.carried_items])
        print("Room contents:", [(room.value, [item.value for item in items]) for room, items in state.room_contents.items()])
        print("Room content values:", [(room.value, state.room_value(room)) for room in state.room_contents])
        print("Charge", state.robot.battery)

In [8]:
ITEM_WEIGHTS = {
    Item.RUSTY_KEY : 0,
    Item.BUCKET : 2,
    Item.SUITCASE : 4,
    Item.SCREWDRIVER : 1,
    Item.SLEDGE_HAMMER : 5,
    Item.ANVIL : 12,
    Item.SAW : 2,
    Item.GOLD_BAR : 7,
    Item.SILVER_BAR : 3,
    Item.DIAMOND_RING : 1,
    Item.OVEN_MITS : 0,
    Item.HOT_SOUP : 3,
    Item.LASAGNA : 3
}

ITEM_VALUES = {
    Item.RUSTY_KEY : -10,
    Item.BUCKET : -2,
    Item.SUITCASE : 1,
    Item.SCREWDRIVER : 2,
    Item.SLEDGE_HAMMER : 5,
    Item.ANVIL : 16,
    Item.SAW : -1,
    Item.GOLD_BAR : 35,
    Item.SILVER_BAR : 19,
    Item.DIAMOND_RING : 43,
    Item.OVEN_MITS : 0,
    Item.HOT_SOUP : 2,
    Item.LASAGNA : 6
}

# Helper function
def reverse_dependency_map(dependent_items):
    reverse_map = {}

    for item, dependents in dependent_items.items():
        for dependent in dependents:
            if dependent not in reverse_map:
                reverse_map[dependent] = set()
            reverse_map[dependent].add(item)

    return reverse_map

# One item in set must be in inventory before item can be picked up
# Item cannot be removed if any items in set still being carried
DEPENDENT_ITEMS = {
    Item.OVEN_MITS : {Item.HOT_SOUP, Item.LASAGNA},
}

REQUIRED_ITEMS = reverse_dependency_map(DEPENDENT_ITEMS)

### Heuristics

In [9]:
def HEURISTIC_0(state : State) -> int:
    """
    Prioritizes being in a room that is has some goal requirement
    """
    return 1 - int(state.robot.location in [goal.room for goal in state.goal_params])


def HEURISTIC_1(state : State) -> int:

    """
    Heuristic that encourages dropping items in rooms
    with low current value (and require a higher one)
    """

    current_room = state.robot.location
    carried_items = state.robot.carried_items

    if not carried_items:
        return 1000
    
    current_value = state.room_value(current_room)

    for goal in state.goal_params:
        if goal.room == current_room:
            # Discourage this move if room's value is already satisfied
            if current_value >= goal.desired_value:
                return 1000

            value_increase = sum(ITEM_VALUES[item] for item in carried_items)
            potential_value = current_value + value_increase

            # Prioritize reaching desired value with smalled value increase
            return max(0, goal.desired_value - potential_value)

    # If room has no goal, prioritize minimizinzing the value increase
    return current_value


def HEURISTIC_2(state : State) -> int:
    """
    Heuristic that prioritizes dropping items in rooms
    that have the fewest missing required items
    """

    carried_items = state.robot.carried_items

    if not carried_items:
        return 1000

    room_missing_items = {}

    for goal in state.goal_params:
        missing_items = goal.contents - set(carried_items)
        if missing_items:
            room_missing_items[goal.room] = len(missing_items)

    if not room_missing_items:
        return 1000

    # Find room that has the fewest missing required items
    best_value = min(room_missing_items.values())

    return best_value


def HEURISTIC_3(state : State) -> int:
    """
    Heuristic that prioritizes picking up items with 
    more value.
    """

    best_item_value = -sys.maxsize

    for item in state.room_contents[state.robot.location]:
        best_item_value = max(best_item_value, ITEM_VALUES[item])

    return best_item_value


def HEURISTIC_4(state : State) -> int:
    """
    Heuristic that prioritizes picking up dependent items
    """

    carried_items = state.robot.carried_items
    carried_dependent_items = []

    for item in carried_items:
        if item in DEPENDENT_ITEMS:
            carried_dependent_items.append(item)

    return len(carried_dependent_items)


global HEURISTIC_WEIGHTS
def HEURISTIC_TOTAL(state : State) -> int:
    return (HEURISTIC_WEIGHTS[0] * HEURISTIC_0(state) + 
            HEURISTIC_WEIGHTS[1] * HEURISTIC_1(state) + 
            HEURISTIC_WEIGHTS[2] * HEURISTIC_2(state) + 
            HEURISTIC_WEIGHTS[3] * HEURISTIC_3(state) + 
            HEURISTIC_WEIGHTS[4] * HEURISTIC_4(state))


def COST(path, state):
    return len(path)

### Problems

In [10]:
def RW_PROBLEM() -> RobotWorker:
    room_contents = {
        Room.WORKSHOP: {Item.GOLD_BAR, Item.DIAMOND_RING},
        Room.KITCHEN: {Item.LASAGNA},
        Room.BEDROOM: set(),
        Room.CHARGING_ROOM: set(),
        Room.STORE_ROOM: {Item.OVEN_MITS, Item.SLEDGE_HAMMER}
    }

    doors = [
        Door(Room.KITCHEN, Room.WORKSHOP),
        Door(Room.WORKSHOP, Room.CHARGING_ROOM),
        Door(Room.KITCHEN, Room.BEDROOM),
        Door(Room.BEDROOM, Room.STORE_ROOM),
        Door(Room.KITCHEN, Room.STORE_ROOM)
    ]

    robot = Robot(Room.WORKSHOP, [Item.ANVIL], 15, 10, 5)
    initial_state = State(robot, doors, room_contents)
    goal_params = {
        GoalParam(Room.WORKSHOP, {Item.SLEDGE_HAMMER, Item.ANVIL}, 0),
        GoalParam(Room.BEDROOM, {Item.DIAMOND_RING, Item.LASAGNA}, 50)
    }

    return RobotWorker(initial_state, goal_params)

### Testing

In [11]:
results = {}

TEST_CASE = RW_PROBLEM()

results["BF"] = search(TEST_CASE, mode="BF/FIFO", max_nodes=100000, loop_check=True, return_info=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
................................................................
!! Search node limit (100000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =   220436  (includes start)
Nodes discarded by loop_check  =   120435  (100001 distinct states added to queue)
Nodes tested (by goal_test)    =    64753  (all expanded)
Nodes left in queue            =    35247

Time taken = 65.0539 seconds



In [12]:
results["DF"] = search(TEST_CASE, mode="DF/LIFO", max_nodes=100000, loop_check=True, return_info=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=None, heuristic=None
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
..........................
:-)) *SUCCESS* ((-:

Path length = 3696
Goal state is:
Robot location: Bedroom
Robot carrying: ['Oven Mits']
Room contents: [('Workshop', ['Sledge Hammer', 'Anvil']), ('Kitchen', []), ('Bedroom', ['Lasagna', 'Gold Bar', 'Diamond Ring']), ('Charging Room', []), ('Store Room', [])]
Room content values: [('Workshop', 21), ('Kitchen', 0), ('Bedroom', 84), ('Charging Room', 0), ('Store Room', 0)]
Charge 0
The action path to the solution is:
    (<Action.CHANGE_ROOM: 'Change room to'>, <Room.CHARGING_ROOM: 'Charging Room'>)
    (<Action.CHANGE_ROOM: 'Change room to'>, <Room.WORKSHOP: 'Workshop'>)
    (<Action.CHANGE_ROOM: 'Change room to'>, <Room.KITCHEN: 'Kitche

In [ ]:
results["BF_HUERISTIC3"] = search(TEST_CASE, mode="BF/FIFO", max_nodes=1000000, loop_check=True, return_info=True, heuristic=HEURISTIC_3)
results["BF_HUERISTIC3+COST"] = search(TEST_CASE, mode="BF/FIFO", max_nodes=1000000, loop_check=True, return_info=True, heuristic=HEURISTIC_3, cost=COST)

results["DF_HUERISTIC3"] = search(TEST_CASE, mode="DF/LIFO", max_nodes=1000000, loop_check=True, return_info=True, heuristic=HEURISTIC_3)
results["DF_HUERISTIC3+COST"] = search(TEST_CASE, mode="DF/LIFO", max_nodes=1000000, loop_check=True, return_info=True, heuristic=HEURISTIC_3, cost=COST)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=HEURISTIC_3
Max search nodes: 1000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................

In [ ]:
import pandas as pd

# Flatten the nested dictionaries into a DataFrame


def flatten_results(results):
    rows = []

    for test_name, data in results.items():
        row = {"test_name": test_name}

        # Flatten args
        row.update(data.get("args", {}))

        # Flatten result
        result = data.get("result", {})
        row.update({
            "termination_condition": result.get("termination_condition"),
            "goal_state": result.get("goal_state"),
            "path_length": result.get("path_length")
        })

        # Flatten search stats
        stats = data.get("search_stats", {})
        row.update({
            "nodes_generated": stats.get("nodes_generated"),
            "nodes_tested": stats.get("nodes_tested"),
            "nodes_discarded": stats.get("nodes_discarded"),
            "distinct_states_seen": stats.get("distinct_states_seen"),
            "nodes_left_in_queue": stats.get("nodes_left_in_queue"),
            "time_taken": stats.get("time_taken")
        })

        rows.append(row)

    return pd.DataFrame(rows)


# Convert to DataFrame
df = flatten_results(results)

# Display the DataFrame
print(df.head())

global i
i+=1
df.to_csv(f"result_{i}.csv",index=False)

  test_name      problem     mode  max_nodes  loop_check  randomise  cost  \
0        BF  RobotWorker  BF/FIFO     100000        True      False  None   
1        DF  RobotWorker  DF/LIFO     100000        True      False  None   

  heuristic  dots termination_condition  \
0      None  True   NODE_LIMIT_EXCEEDED   
1      None  True      GOAL_STATE_FOUND   

                                          goal_state  path_length  \
0                                               None          NaN   
1  ("(<Room.BEDROOM: 'Bedroom'>, [<Item.OVEN_MITS...       3696.0   

   nodes_generated  nodes_tested  nodes_discarded  distinct_states_seen  \
0           220436         64753           120435                100001   
1            63793         26875            28340                 35453   

   nodes_left_in_queue  time_taken  
0                35247   65.053939  
1                 8578   16.794039  


In [ ]:

WEIGHTS = [[1, 0, 0, 0, 0],  # Only HEURISTIC_0
           [0, 1, 0, 0, 0],  # Only HEURISTIC_1
           [0, 0, 1, 0, 0],  # Only HEURISTIC_2
           [0, 0, 0, 1, 0],  # Only HEURISTIC_3
           [0, 0, 0, 0, 1],  # Only HEURISTIC_4
           [1, 1, 1, 1, 1],  # Balanced Weights
           [10, 1, 2, 1, 4]]  # Inferred Weights

rw_3_res = {}
prob = RW_PROBLEM_3()

rw_3_res["BF"] = search(test_case(), mode="BF/FIFO", max_nodes=100000, loop_check=True, return_info=True)
rw_3_res["DF"] = search(test_case(), mode="DF/LIFO", max_nodes=100000, loop_check=True, return_info=True)
for i, WEIGHT in enumerate(WEIGHTS):
    HEURISTIC_WEIGHTS = WEIGHT
    print("\nWEIGHTS: ",f"({i})", WEIGHT)
    rw_3_res[test_case.__name__+"_BF_HUERISTIC_COMBIN_"+str(i)] = search(prob, mode="BF/FIFO", max_nodes=1000000, loop_check=True, return_info=True, heuristic=HEURISTIC_TOTAL)
    rw_3_res[test_case.__name__+"_BF_HUERISTIC_COMBIN_"+str(i)+"_COST"] = search(prob, mode="BF/FIFO", max_nodes=1000000, loop_check=True, return_info=True, heuristic=HEURISTIC_TOTAL, cost=COST)
    
    rw_3_res[test_case.__name__+"_DF_HUERISTIC_COMBIN_"+str(i)] = search(prob, mode="DF/LIFO", max_nodes=1000000, loop_check=True, return_info=True, heuristic=HEURISTIC_TOTAL)
    rw_3_res[test_case.__name__+"_DF_HUERISTIC_COMBIN_"+str(i)+"_COST"] = search(prob, mode="DF/LIFO", max_nodes=1000000, loop_check=True, return_info=True, heuristic=HEURISTIC_TOTAL, cost=COST)

In [ ]:
df[0:30][["test_name","path_length","nodes_generated"]]